## Hybrid Search

In [ ]:
import os
from dotenv import load_dotenv

# assuming notebook is inside src/, and .env is one level up
load_dotenv("../.env")
api_key = os.getenv("PINECONE_DB_API_KEY")


In [ ]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone"

## initialize the Pinecone client
pc = Pinecone(api_key=api_key)

#create the index
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,  # dimensionality of dense model beacuse we are using all-MiniLM-L6-v2 and this model has 384 dimensions
        metric="dotproduct",  # sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1"), # choose the cloud and region closest to me
    )

In [ ]:
# index is the interface to the Pinecone index that we can use to upsert and query
index=pc.Index(index_name)
index

### vector embedding and sparse matrix

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]=os.getenv("HUGGINGFACEHUB_API_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

In [ ]:
# BM25Encoder is used to encode the text into sparse vectors
# uses TF-IDF under the hood by default
from pinecone_text.sparse import BM25Encoder

bm25_encoder=BM25Encoder().default()
bm25_encoder

In [ ]:
sentences=[
    "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",

]

## tfidf values on these sentence and fit the encoder
bm25_encoder.fit(sentences)

In [ ]:
## store the values to a json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

In [ ]:
# create the retriever
# this retriever has both sparse and dense capabilities
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [ ]:
retriever

In [ ]:
# add some texts to the index
retriever.add_texts(
    [
    "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",
]
)

In [ ]:
retriever.invoke("What city did i visit last")